In [1]:
import pandas as pd

In [2]:
#Load data
train_ratings = pd.read_csv("/content/drive/MyDrive/BIA project/data/train_full.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_ratings.head()

,customer_id,gender,status_x,verified_x,created_at_x,updated_at_x,location_number,location_type,latitude_x,longitude_x,id,authentication_id,latitude_y,longitude_y,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,commission,is_akeed_delivering,discount_percentage,status_y,verified_y,rank,language,vendor_rating,sunday_from_time1,sunday_to_time1,sunday_from_time2,sunday_to_time2,monday_from_time1,monday_to_time1,monday_from_time2,monday_to_time2,tuesday_from_time1,tuesday_to_time1,tuesday_from_time2,tuesday_to_time2,wednesday_from_time1,wednesday_to_time1,wednesday_from_time2,wednesday_to_time2,thursday_from_time1,thursday_to_time1,thursday_from_time2,thursday_to_time2,friday_from_time1,friday_to_time1,friday_from_time2,friday_to_time2,saturday_from_time1,saturday_to_time1,saturday_from_time2,saturday_to_time2,primary_tags,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at_y,updated_at_y,device_type,display_orders,location_number_obj,id_obj,CID X LOC_NUM X VENDOR,target
0,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,4,118597.0,-0.5884,0.7544,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,-,15,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,"{""primary_tags"":""4""}",1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1,0,4,TCHWPBT X 0 X 4,0
1,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,13,118608.0,-0.4717,0.7446,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,-,14,0.0,Yes,0.0,1.0,1,11,EN,4.7,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,"{""primary_tags"":""7""}",1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1,0,13,TCHWPBT X 0 X 13,0
2,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,20,118616.0,-0.4075,0.6436,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,-,19,0.0,Yes,0.0,1.0,1,1,EN,4.5,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,"{""primary_tags"":""71""}",1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1,0,20,TCHWPBT X 0 X 20,0
3,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,23,118619.0,-0.5854,0.7540,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,-,16,0.0,Yes,0.0,1.0,1,11,EN,4.5,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,"{""primary_tags"":""46""}",1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1,0,23,TCHWPBT X 0 X 23,0
4,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,28,118624.0,0.4807,0.5527,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,-,10,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,"{""primary_tags"":""32""}",1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1,0,28,TCHWPBT X 0 X 28,0


In [4]:
train_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5802400 entries, 0 to 5802399
Data columns (total 73 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   customer_id             object 
 1   gender                  object 
 2   status_x                int64  
 3   verified_x              int64  
 4   created_at_x            object 
 5   updated_at_x            object 
 6   location_number         int64  
 7   location_type           object 
 8   latitude_x              float64
 9   longitude_x             float64
 10  id                      int64  
 11  authentication_id       float64
 12  latitude_y              float64
 13  longitude_y             float64
 14  vendor_category_en      object 
 15  vendor_category_id      float64
 16  delivery_charge         float64
 17  serving_distance        float64
 18  is_open                 float64
 19  OpeningTime             object 
 20  OpeningTime2            object 
 21  prepration_time         int64  

In [5]:
#creating new df with customer_id, id, vendor_rating

train_df = train_ratings[['customer_id','id','vendor_rating']]

In [6]:
train_df.head()

,customer_id,id,vendor_rating
0,TCHWPBT,4,4.4
1,TCHWPBT,13,4.7
2,TCHWPBT,20,4.5
3,TCHWPBT,23,4.5
4,TCHWPBT,28,4.4


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5802400 entries, 0 to 5802399
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_id    object 
 1   id             int64  
 2   vendor_rating  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 132.8+ MB


In [8]:
#check for NA or inf values 
print(train_df.isna().sum())


customer_id      0
id               0
vendor_rating    0
dtype: int64


In [9]:
#create new column all for new key to delete duplicate rows
cols = ['customer_id', 'id']
train_df['all'] = train_df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
train_df.drop_duplicates(['all'], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3452300 entries, 0 to 5802399
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_id    object 
 1   id             int64  
 2   vendor_rating  float64
 3   all            object 
dtypes: float64(1), int64(1), object(2)
memory usage: 131.7+ MB


In [12]:
#One hot encoding of customer id
from sklearn import preprocessing

customer_id_1e = preprocessing.LabelEncoder()

customer_id_1e.fit(train_df['customer_id'])

print("Number of unique customers: ", str(len(customer_id_1e.classes_)))
print("Number of unique vendors: ", str(train_df['id'].nunique()))


Number of unique customers:  34523
Number of unique vendors:  100


In [13]:
#Translate customer id to number form

train_df_1 = train_df
train_df_1.customer_id = customer_id_1e.transform(train_df.customer_id)
train_df_1.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,customer_id,id,vendor_rating,all
0,28177,4,4.4,TCHWPBT_4
1,28177,13,4.7,TCHWPBT_13
2,28177,20,4.5,TCHWPBT_20
3,28177,23,4.5,TCHWPBT_23
4,28177,28,4.4,TCHWPBT_28


In [14]:
train_df_1.drop('all',axis=1)

,customer_id,id,vendor_rating
0,28177,4,4.4
1,28177,13,4.7
2,28177,20,4.5
3,28177,23,4.5
4,28177,28,4.4
...,...,...,...
5802395,20387,849,4.1
5802396,20387,855,4.2
5802397,20387,856,4.3
5802398,20387,858,4.2


In [15]:
train_df_2 = train_df_1[['customer_id', 'id', 'vendor_rating']]
train_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3452300 entries, 0 to 5802399
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_id    int64  
 1   id             int64  
 2   vendor_rating  float64
dtypes: float64(1), int64(2)
memory usage: 105.4 MB


In [16]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 198 kB 36.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4c9163077c77e2e82e29d3b7880496b621ece646ae0792b85ac2da6868b9c8ce
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [17]:
#creating spark session
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

from pyspark.mllib.recommendation import *

sc = SparkContext.getOrCreate()
sc.stop()
sc = SparkContext('local', 'Recommender')
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [18]:
#convert pandas dataframe into rdd
train_df_rdd = sqlContext.createDataFrame(train_df_2).rdd

In [19]:
train_df_rdd.take(5)

[Row(customer_id=28177, id=4, vendor_rating=4.4),
 Row(customer_id=28177, id=13, vendor_rating=4.7),
 Row(customer_id=28177, id=20, vendor_rating=4.5),
 Row(customer_id=28177, id=23, vendor_rating=4.5),
 Row(customer_id=28177, id=28, vendor_rating=4.4)]

In [20]:
#Splitting data into train, val and test rdds

train_rdd, val_rdd, test_rdd = train_df_rdd.randomSplit([6,2,2],42)
val_for_pred_rdd = val_rdd.map(lambda x: (x[0], x[1]))
test_for_pred_rdd = test_rdd.map(lambda x: (x[0], x[1]))

print("Train RDD\n", train_rdd.take(5),"\n")
print("Validation for prediction RDD\n", val_for_pred_rdd.take(5),"\n")
print("Test for prediction RDD\n", test_for_pred_rdd.take(5),"\n")

Train RDD
 [Row(customer_id=28177, id=4, vendor_rating=4.4), Row(customer_id=28177, id=13, vendor_rating=4.7), Row(customer_id=28177, id=20, vendor_rating=4.5), Row(customer_id=28177, id=23, vendor_rating=4.5), Row(customer_id=28177, id=28, vendor_rating=4.4)] 

Validation for prediction RDD
 [(28177, 33), (28177, 76), (28177, 86), (28177, 92), (28177, 115)] 

Test for prediction RDD
 [(28177, 66), (28177, 75), (28177, 81), (28177, 85), (28177, 90)] 



In [21]:
import math

seed = 42
iterations = 10
lambdas = [0.001, 0.01, 0.1]
ranks = [16, 32, 64]
errors = [0 for x in range(len(ranks)*len(lambdas))]
err_index = 0

for lambda_ in lambdas:
  for rank in ranks:
    model = ALS.train(train_rdd, rank, seed = seed, iterations=iterations, lambda_=lambda_)
    predictions = model.predictAll(val_for_pred_rdd).map(lambda r: ((r[0], r[1]), r[2]))
    ratings_and_preds = val_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(ratings_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err_index] = error
    err_index+=1
    print('For rank {0} at lambda: {1}, the RMSE is {2}'.format(rank, lambda_, error))





For rank 16 at lambda: 0.001, the RMSE is 0.022544435005567895
For rank 32 at lambda: 0.001, the RMSE is 0.006985169903862695
For rank 64 at lambda: 0.001, the RMSE is 0.00351967729579193
For rank 16 at lambda: 0.01, the RMSE is 0.13079477295186043
For rank 32 at lambda: 0.01, the RMSE is 0.0765385283333102
For rank 64 at lambda: 0.01, the RMSE is 0.06067927227968845
For rank 16 at lambda: 0.1, the RMSE is 0.19778849495417283
For rank 32 at lambda: 0.1, the RMSE is 0.206145507893568
For rank 64 at lambda: 0.1, the RMSE is 0.19973748644177378


In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.recommendation import ALS

In [23]:
als = ALS(userCol='customer_id', itemCol='id', ratingCol='vendor_rating', nonnegative=True, implicitPrefs = False, coldStartStrategy='drop')

In [24]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("ALSrecommender")\
    .getOrCreate()

In [25]:
new_train_rdd = spark.createDataFrame(train_df_2)

In [26]:
(training, val, test) = new_train_rdd.randomSplit([0.6,0.2,0.2],42)

In [27]:
best_als_model = ALS(maxIter=10, regParam=0.001, userCol='customer_id', itemCol='id', ratingCol="vendor_rating", coldStartStrategy="drop", rank=64)

In [28]:
final_model = best_als_model.fit(training)

In [29]:
predictions = final_model.transform(test)

In [30]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="vendor_rating", predictionCol="prediction")


In [31]:
rmse = evaluator.evaluate(predictions)

In [32]:
print("Root mean squared error = " + str(rmse))

Root mean squared error = 0.0022238419206284626


In [33]:
#Generate top 10 vendor recommendations for each customer
custRecs = final_model.recommendForAllUsers(10)

#Generate top 10 customer recommendations for each vendor
vendRecs = final_model.recommendForAllItems(10)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [34]:
## Generate top 10 vendor recommendations for a specified set of customers

customers = new_train_rdd.select(best_als_model.getUserCol()).distinct().limit(3)
custSubsetRecs = final_model.recommendForUserSubset(customers, 10)

# Generate top 10 customer recommendations for a specified set of vendors

vendors = new_train_rdd.select(best_als_model.getItemCol()).distinct().limit(3)
vendSubSetRecs = final_model.recommendForItemSubset(vendors, 10)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [35]:
custRecs.show()


+-----------+--------------------+
|customer_id|     recommendations|
+-----------+--------------------+
|          0|[{310, 4.797579},...|
|          1|[{115, 4.797552},...|
|          2|[{115, 4.7975435}...|
|          3|[{310, 4.7975245}...|
|          4|[{310, 4.7975435}...|
|          5|[{310, 4.7975516}...|
|          6|[{310, 4.7975287}...|
|          7|[{310, 4.797538},...|
|          8|[{310, 4.797517},...|
|          9|[{310, 4.797567},...|
|         10|[{310, 4.797529},...|
|         11|[{310, 4.7975726}...|
|         12|[{310, 4.7975516}...|
|         13|[{115, 4.797581},...|
|         14|[{310, 4.79756}, ...|
|         15|[{310, 4.7975583}...|
|         16|[{310, 4.797527},...|
|         17|[{115, 4.797552},...|
|         18|[{310, 4.7975225}...|
|         19|[{310, 4.7975545}...|
+-----------+--------------------+
only showing top 20 rows



In [36]:
vendRecs.show()

+---+--------------------+
| id|     recommendations|
+---+--------------------+
|300|[{34246, 4.397847...|
| 20|[{34246, 4.497798...|
|250|[{34246, 4.497798...|
|310|[{34246, 4.797647...|
|160|[{34246, 4.297896...|
| 90|[{34246, 4.397847...|
|110|[{34246, 4.597747...|
|180|[{34246, 3.798144...|
| 81|[{34246, 3.798144...|
|271|[{34246, 4.497798...|
|191|[{34246, 4.497798...|
|231|[{34246, 4.297896...|
|161|[{34246, 4.197945...|
|391|[{34246, 4.197945...|
|221|[{34246, 4.197945...|
|201|[{34246, 3.998045...|
|681|[{34246, 4.397847...|
|841|[{34246, 3.998045...|
|401|[{34246, 4.497798...|
|192|[{34246, 4.297896...|
+---+--------------------+
only showing top 20 rows



In [37]:
custSubsetRecs.show()

+-----------+--------------------+
|customer_id|     recommendations|
+-----------+--------------------+
|      19771|[{115, 4.7974863}...|
|      18196|[{310, 4.797581},...|
|      13638|[{115, 4.797519},...|
+-----------+--------------------+



In [38]:
vendSubSetRecs.show()

+---+--------------------+
| id|     recommendations|
+---+--------------------+
|191|[{34246, 4.497798...|
|243|[{34246, 4.397847...|
|113|[{34246, 4.697697...|
+---+--------------------+

